In [9]:
from random import randrange, random
from time import strptime, mktime
import requests

idRand = None                    # ідентефікатор аккаунта
account = None                   # для Bet


def createAccount():
    global idRand, account
    for i in range(10):  # десять спроб створити account
        idRand = randrange(30000) 
        response = requests.get('http://95.217.177.249/casino/createacc',
                                        params=dict(id=idRand))
        account = response.json()
        print(account)
        if "error" not in account:
            break
    else:
        idRand = None
        account = None


def makeBet(mode, betAmount, number):
    response = requests.get(f'http://95.217.177.249/casino/play{mode}',
                        params=dict(id=idRand, bet=betAmount, number=number))
    bet = response.json()
    print(bet)
    return bet.get('realNumber')

In [10]:
modulus = 2 ** 32                # константа для LCG


def modInv(a, n): # для LCG
    i, v, d = n, 0, 1
    while a > 0:
        i, t, a = a, *divmod(i, a)
        v, d = d, v - t * d
    return (v + n) % n

def modInv(a, n): # для LCG
    i, v, d = n, 0, 1
    while a > 0:
        i, t, a = a, *divmod(i, a)
        v, d = d, v - t * d
    return (v + n) % n


def findVariables(numbers): # для LCG
    diff1 = numbers[2] - numbers[1]
    diff2 = numbers[1] - numbers[0]
    inverse = modInv(diff2, modulus)
    a = (diff1 * inverse) % modulus
    c = (numbers[1] - a * numbers[0]) % modulus
    print(f"{a = }")
    print(f"{c = }")
    return a, c


def lcg_next(last, a, c):
    return (a * last + c) % modulus


def LcgCrack():
    numbers = []
    for i in range(3):
        numbers.append(makeBet("Lcg", 1, 1))
    a, c = findVariables(numbers)
    next = lcg_next(numbers[2], a, c)  # беремо останній номер і вкладаємо його в рівняння і отримуємо наступне число
    print(f"Передбачене наступне число = {next}")
    makeBet("Lcg", 500, next)
    next = lcg_next(next, a, c)
    print(f"Передбачене наступне число = {next}")
    makeBet("Lcg", 501, next)


In [11]:
# МЕ19937
N = 624                          # різні числа які формують стан
M = 397                          

MATRIX_A = 0x9908B0DF            
UPPER_MASK = 0x80000000          
LOWER_MASK = 0X7FFFFFFF          
DEFAULT_SEED = 5489              

mt = [0 for i in range(N + 1)]   #  масив станів, н+1 означає що він не згенерований не заповнений
mti = N + 1                      # розмір масиву

def tempering(y):  # зсуви для МЕ19937
    y ^= y >> 11                 
    y ^= y << 7  & 0x9d2c5680
    y ^= y << 15 & 0xefc60000
    y ^= y >> 18
    return y


def unTempering(y): # реверс зсувів для МTBetter
    y ^= y >> 18
    y ^= y << 15 & 0xefc60000
    y ^= (y << 7 & 0x9d2c5680 ^
        y << 14 & 0x94284000 ^
        y << 21 & 0x14200000 ^
        y << 28 & 0x10000000)
    y ^= y >> 11 ^ y >> 22
    return y


# ф-ія яка заповнює масив станів використовуючи сід
def init_genrand_1(s=DEFAULT_SEED): 
    global mt, mti
    ustates = [s & 0xffffffff]
    for i in range(1, N):
        ustates.append((1812433253 * (ustates[i - 1] ^
                        ustates[i - 1] >> 30) + i) & 0xffffffff)
    mt = ustates
    mti = N # чому тут N ?????

# ф-ія яка заповнює масив станів використовуючи стан коли немає сіда 
def init_genrand_n(states):
    global mt, mti
    ustates = []
    for state in states:
        ustates.append(unTempering(state))
    mt = ustates
    mti = N 

# функція яка з 624 вибирає наступне випадкове значення 
def genrand_int32():
    global mt, mti
    y = 0
    mag01 = [0, MATRIX_A]
    if mti >= N:
        if mti == N + 1:
            init_genrand_1()
        for kk in range(N - M):
            y = (mt[kk] & UPPER_MASK) | (mt[kk + 1] & LOWER_MASK)
            mt[kk] = mt[kk + M] ^ (y >> 1) ^ mag01[y & 0x1]
        for kk in range(kk, N - 1):
            y = (mt[kk] & UPPER_MASK) | (mt[kk + 1] & LOWER_MASK)
            mt[kk] = mt[kk - 227] ^ (y >> 1) ^ mag01[y & 0x1]
        y = (mt[N - 1] & UPPER_MASK) | (mt[0] & LOWER_MASK)
        mt[N - 1] = mt[M - 1] ^ (y >> 1) ^ mag01[y & 0x1]
        mti = 0
    y = mt[mti]
    mti += 1
    y = tempering(y)  # числа нерівномірно розподілені і нам треба зробити їх рівномірнішими таким чином
    return y


def MTCrack():
    global mt, mti
    mt = [0 for i in range(N + 1)] 
    mti = N + 1
    number = makeBet("Mt", 1, 1)
    print(f'{number = }')
    Time = account['deletionTime']
    seed = int(mktime(strptime(Time.rsplit('.', 1)[0], '%Y-%m-%dT%H:%M:%S'))) - 3600
    predictedNumber = -1
    while predictedNumber != number:
        init_genrand_1(seed)
        predictedNumber = genrand_int32()
        seed += 1
    for i in range(10):
        makeBet("Mt", 110, genrand_int32())


def BetterMTCrack():
    global mt, mti
    mt = [0 for i in range(N + 1)] 
    mti = N + 1
    numbers = []
    for i in range(N):
        numbers.append(makeBet("BetterMt", 1, 1))
    init_genrand_n(numbers)
    for i in range(10):
        predictedNumber = genrand_int32()
        print(f'{predictedNumber = }')
        makeBet("BetterMt", 110, predictedNumber)


In [12]:
createAccount()
account and LcgCrack()

{'id': '12955', 'money': 1000, 'deletionTime': '2021-12-26T21:20:47.7429402Z'}
{'message': 'You lost this time', 'account': {'id': '12955', 'money': 999, 'deletionTime': '2021-12-26T21:20:47.7429402Z'}, 'realNumber': 1669057394}
{'message': 'You lost this time', 'account': {'id': '12955', 'money': 998, 'deletionTime': '2021-12-26T21:20:47.7429402Z'}, 'realNumber': 2062136361}
{'message': 'You lost this time', 'account': {'id': '12955', 'money': 997, 'deletionTime': '2021-12-26T21:20:47.7429402Z'}, 'realNumber': 806776692}
a = 1664525
c = 1013904223
Передбачене наступне число = 2152649795
{'type': 'https://tools.ietf.org/html/rfc7231#section-6.5.1', 'title': 'One or more validation errors occurred.', 'status': 400, 'traceId': '|9c33cc4c-4f1a57ad4f8c317b.', 'errors': {'number': ["The value '2152649795' is not valid."]}}
Передбачене наступне число = 3817696454
{'type': 'https://tools.ietf.org/html/rfc7231#section-6.5.1', 'title': 'One or more validation errors occurred.', 'status': 400, '

In [14]:
createAccount()
account and MTCrack()

{'id': '7400', 'money': 1000, 'deletionTime': '2021-12-26T21:22:14.8828521Z'}
{'message': 'You lost this time', 'account': {'id': '7400', 'money': 999, 'deletionTime': '2021-12-26T21:22:14.8828521Z'}, 'realNumber': 3657604923}
number = 3657604923
{'message': "Yay! It's different from the first one: https://docs.google.com/document/d/1_W00GZXLNTk6BML6jEaAJDqwMVjaQUv5WL1DCW7ipy4/edit", 'account': {'id': '7400', 'money': 110889, 'deletionTime': '2021-12-26T21:22:14.8828521Z'}, 'realNumber': 1587449471}
{'message': "Yay! It's different from the first one: https://docs.google.com/document/d/1_W00GZXLNTk6BML6jEaAJDqwMVjaQUv5WL1DCW7ipy4/edit", 'account': {'id': '7400', 'money': 220779, 'deletionTime': '2021-12-26T21:22:14.8828521Z'}, 'realNumber': 2893604404}
{'message': "Yay! It's different from the first one: https://docs.google.com/document/d/1_W00GZXLNTk6BML6jEaAJDqwMVjaQUv5WL1DCW7ipy4/edit", 'account': {'id': '7400', 'money': 330669, 'deletionTime': '2021-12-26T21:22:14.8828521Z'}, 'real

In [13]:
createAccount()
account and BetterMTCrack()

{'error': 'Id already in use'}
{'id': '13499', 'money': 1000, 'deletionTime': '2021-12-26T21:20:48.5729218Z'}
{'message': 'You lost this time', 'account': {'id': '13499', 'money': 999, 'deletionTime': '2021-12-26T21:20:48.5729218Z'}, 'realNumber': 3263397629}
{'message': 'You lost this time', 'account': {'id': '13499', 'money': 998, 'deletionTime': '2021-12-26T21:20:48.5729218Z'}, 'realNumber': 1740453520}
{'message': 'You lost this time', 'account': {'id': '13499', 'money': 997, 'deletionTime': '2021-12-26T21:20:48.5729218Z'}, 'realNumber': 3285792552}
{'message': 'You lost this time', 'account': {'id': '13499', 'money': 996, 'deletionTime': '2021-12-26T21:20:48.5729218Z'}, 'realNumber': 2871794300}
{'message': 'You lost this time', 'account': {'id': '13499', 'money': 995, 'deletionTime': '2021-12-26T21:20:48.5729218Z'}, 'realNumber': 3275629508}
{'message': 'You lost this time', 'account': {'id': '13499', 'money': 994, 'deletionTime': '2021-12-26T21:20:48.5729218Z'}, 'realNumber': 32

createAccount()
account and MTCrack()